In [172]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5py
import pandas as pd
#sess = tf.InteractiveSession()

In [173]:
def load_dataset():
    train_dataset = h5py.File('signs_dataset/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('signs_dataset/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [180]:
def tf_var_normInit(shape):
    '''Helper function to create a weight variable initialized with
    a normal distribution

    Parameters
    ----------
    shape : list
        Size of weight variable
    '''
    initial = tf.random_normal(shape, mean=0.0, stddev=0.01,dtype=tf.float32)
    return tf.Variable(initial)

In [181]:
def conv_params(lx_filter_size,ch_N,lx_filter_Num):
    '''Helper function to create weights and biases

    Parameters
    ----------
    lx_filter_size : filter of (size x size) of layer 'x'
    
    ch_N : channel size of the filter i.e. (size x size x ch_N)
    
    lx_filter_Num : No of filters required for filter of layer 'x'
    '''
    
    W_conv = tf_var_normInit([lx_filter_size, lx_filter_size, ch_N, lx_filter_Num])
    B_conv = tf_var_normInit([lx_filter_Num])
    return W_conv, B_conv

In [182]:
LOGDIR = "/home/vishnu/Dropbox/intel_works/ipython_notebooks/tf_tests/logs"

#loading the dataset
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()
#index = 1079
#plt.imshow(train_set_x_orig[index])
print(train_set_x_orig.shape)
print("No. of Training samples (batch): %d"%train_set_x_orig.shape[0])
print("No. of Test samples: %d"%test_set_x_orig.shape[0])
print("No. of gestures: %d"%classes.size)
print("Image format (Len, Wid, Chan): %d * %d * %d"%(train_set_x_orig.shape[1],train_set_x_orig.shape[2],train_set_x_orig.shape[3]))

(1080, 64, 64, 3)
No. of Training samples (batch): 1080
No. of Test samples: 120
No. of gestures: 6
Image format (Len, Wid, Chan): 64 * 64 * 3


In [183]:
x = tf.placeholder(tf.float32, [None, train_set_x_orig.shape[1],train_set_x_orig.shape[2],train_set_x_orig.shape[3]])
y = tf.placeholder(tf.float32, [None, classes.size])

x = tf.cast(x,tf.float32) # casting as the original was uint8

In [184]:
#conv1 layer to convert the input to Lenet format
with tf.name_scope('conv1_1'):
    ch_X = train_set_x_orig.shape[3]
    l1_filter_size = 3 # filter size of n*n for convolution
    l1_filter_Num = 3 # No. of filters required

    W_conv1, B_conv1 = conv_params(l1_filter_size,ch_X,l1_filter_Num)
    print(W_conv1.get_shape())

    # %% Now we can build a graph which does the first layer of convolution:
    # we define our stride as batch x height x width x channels
    # instead of pooling, we use strides of 2 and more layers
    # with smaller filters.
    h_conv1 = tf.nn.relu(
        tf.nn.conv2d(
            input=x,
            filter=W_conv1,
            strides=[1, 2, 2, 1],
            padding='SAME') 
        + B_conv1,
        name="h_conv1")
    
    tf.summary.histogram("weights", W_conv1)
    tf.summary.histogram("biases", B_conv1)
    tf.summary.histogram("activations", h_conv1)

(3, 3, 3, 3)


In [185]:
#conv2 for adhering to Lenet
with tf.name_scope('conv1_2'):
    ch_1 = tf.cast(h_conv1.shape[3],tf.int32)
    l2_filter_size = 5 # filter size of n*n for convolution
    l2_filter_Num = 6 # No. of filters required

    W_conv2, B_conv2 = conv_params(l2_filter_size,ch_1,l2_filter_Num)

    h_conv2 = tf.nn.relu(
        tf.nn.conv2d(
            input=h_conv1,
            filter=W_conv2,
            strides=[1, 1, 1, 1],
            padding='VALID') 
        + B_conv2,
        name="h_conv2")
    
    tf.summary.histogram("weights", W_conv2)
    tf.summary.histogram("biases", B_conv2)
    tf.summary.histogram("activations", h_conv2)

In [186]:
#avg pooling layer
with tf.name_scope('pool1'):
    ch_2 = tf.cast(h_conv2.shape[3],tf.int32)
    k_size = [1,2,2,1] #filter size
    h_conv2_pool = tf.nn.avg_pool(h_conv2,k_size,strides=[1, 2, 2, 1],padding='VALID',name="h_conv2_pool")

In [187]:
#conv3 for adhering to Lenet
with tf.name_scope('conv2_1'):
    ch_3 = tf.cast(h_conv2_pool.shape[3],tf.int32)
    l3_filter_size = 5 # filter size of n*n for convolution
    l3_filter_Num = 16 # No. of filters required

    W_conv3, B_conv3 = conv_params(l3_filter_size,ch_3,l3_filter_Num)

    h_conv3 = tf.nn.relu(
        tf.nn.conv2d(
            input=h_conv2_pool,
            filter=W_conv3,
            strides=[1, 1, 1, 1],
            padding='VALID') 
        + B_conv3,
        name="h_conv3")
    
    tf.summary.histogram("weights", W_conv3)
    tf.summary.histogram("biases", B_conv3)
    tf.summary.histogram("activations", h_conv3)

In [188]:
#conv4 for adhering to Lenet
with tf.name_scope('conv2_2'):
    ch_4 = tf.cast(h_conv3.shape[3],tf.int32)
    l4_filter_size = 2 # filter size of n*n for convolution
    l4_filter_Num = 16 # No. of filters required

    W_conv4, B_conv4 = conv_params(l4_filter_size,ch_4,l4_filter_Num)

    h_conv4 = tf.nn.relu(
        tf.nn.conv2d(
            input=h_conv3,
            filter=W_conv4,
            strides=[1, 2, 2, 1],
            padding='VALID') 
        + B_conv4,
        name="h_conv4")
    
    tf.summary.histogram("weights", W_conv4)
    tf.summary.histogram("biases", B_conv4)
    tf.summary.histogram("activations", h_conv4)

In [189]:
# %% We'll now reshape so we can connect to a fully-connected layer:
h_conv4_flat = tf.reshape(h_conv4, [-1, 5 * 5 * 16])
print(h_conv4_flat.get_shape())

(?, 400)


In [191]:
# %% Create a fully-connected layer 1:
with tf.name_scope('fc1') as scope:
    n_fc1 = 120
    W_fc1 = tf_var_normInit([5 * 5 * 16, n_fc1])
    b_fc1 = tf_var_normInit([n_fc1])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv4_flat, W_fc1) + b_fc1)
    
    tf.summary.histogram("weights", W_fc1)
    tf.summary.histogram("biases", b_fc1)
    tf.summary.histogram("activations", h_fc1)

In [192]:
# %% Create a fully-connected layer 2:
with tf.name_scope('fc2') as scope:
    n_fc2 = 84
    W_fc2 = tf_var_normInit([n_fc1, n_fc2])
    b_fc2 = tf_var_normInit([n_fc2])
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
    
    tf.summary.histogram("weights", W_fc2)
    tf.summary.histogram("biases", b_fc2)
    tf.summary.histogram("activations", h_fc2)

In [193]:
# %% And finally our softmax layer:
W_fc3 = tf_var_normInit([n_fc2, 6])
b_fc3 = tf_var_normInit([6])
y_pred = tf.nn.softmax(tf.matmul(h_fc2, W_fc3) + b_fc3)
print(y_pred.get_shape())

(?, 6)


In [194]:
# %% Define loss/eval/training functions
with tf.name_scope('cross_entropy'):
    cross_entropy = -tf.reduce_sum(y * tf.log(y_pred))
    
    tf.summary.scalar("cross_entropy", cross_entropy)

with tf.name_scope('train'):    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy)

In [195]:
# %% Monitor accuracy
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    
    tf.summary.scalar("accuracy", accuracy)

In [196]:
summ = tf.summary.merge_all()

# %% We now create a new session to actually perform the initialization the
sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter(LOGDIR)
writer.add_graph(sess.graph)

In [200]:
y_org = np.eye(6)[train_set_y] #converting it into one-hot encoding
y_org = np.reshape(y_org,[y_org.shape[1],y_org.shape[2]])

y_val = np.eye(6)[test_set_y] #converting it into one-hot encoding
y_val = np.reshape(y_val,[y_val.shape[1],y_val.shape[2]])

test_x, test_y = test_set_x_orig, y_val
print(test_set_x_orig.shape)
print(y_val.shape)
# %% We'll train in minibatches and report accuracy
batch_size = 200
n_epochs = 5
for epoch_i in range(n_epochs):
    accHist = []
    accHist2 = []
    for batch_i in range(train_set_x_orig.shape[0] // batch_size):
        batch_xs, batch_ys = train_set_x_orig[(batch_i+batch_i*epoch_i):batch_size,:,:,:], y_org[(batch_i+batch_i*epoch_i):batch_size,:]
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
        
    #acc,s = (sess.run([accuracy,summ],feed_dict={x: test_set_x_orig,y: y_val}))
    #writer.add_summary(s, epoch_i)


(120, 64, 64, 3)
(120, 6)
